In [1]:
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import re

In [5]:
from transformers import pipeline
sentiment1 = pipeline("sentiment-analysis")
sentiment2 = pipeline(model="finiteautomata/bertweet-base-sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/987 [00:00<?, ?B/s]

emoji is not installed, thus not converting emoticons or emojis into text. Please install emoji: pip3 install emoji


In [6]:
train = pd.read_csv("data/train_use.csv")
valid = pd.read_csv("data/valid_use.csv")
test = pd.read_csv("data/test_use.csv")

In [8]:
train_senti1 = sentiment1(list(train['text']))
valid_senti1 = sentiment1(list(valid['text']))
test_senti1 = sentiment1(list(test['text']))

In [9]:
train_senti2 = sentiment2(list(train['text']))
valid_senti2 = sentiment2(list(valid['text']))
test_senti2 = sentiment2(list(test['text']))

In [10]:
train_senti1_posi = [train_senti1[i]['score'] if train_senti1[i]['label'] == 'POSITIVE' else 0 for i in range(len(train_senti1))]
train_senti1_nega = [train_senti1[i]['score'] if train_senti1[i]['label'] == 'NEGATIVE' else 0 for i in range(len(train_senti1))]
valid_senti1_posi = [valid_senti1[i]['score'] if valid_senti1[i]['label'] == 'POSITIVE' else 0 for i in range(len(valid_senti1))]
valid_senti1_nega = [valid_senti1[i]['score'] if valid_senti1[i]['label'] == 'NEGATIVE' else 0 for i in range(len(valid_senti1))]
test_senti1_posi = [test_senti1[i]['score'] if test_senti1[i]['label'] == 'POSITIVE' else 0 for i in range(len(test_senti1))]
test_senti1_nega = [test_senti1[i]['score'] if test_senti1[i]['label'] == 'NEGATIVE' else 0 for i in range(len(test_senti1))]

In [11]:
train_senti2_posi = [train_senti2[i]['score'] if train_senti2[i]['label'] == 'POS' else 0 for i in range(len(train_senti2))]
train_senti2_nega = [train_senti2[i]['score'] if train_senti2[i]['label'] == 'NEG' else 0 for i in range(len(train_senti2))]
train_senti2_neu = [train_senti2[i]['score'] if train_senti2[i]['label'] == 'NEU' else 0 for i in range(len(train_senti2))]
valid_senti2_posi = [valid_senti2[i]['score'] if valid_senti2[i]['label'] == 'POS' else 0 for i in range(len(valid_senti2))]
valid_senti2_nega = [valid_senti2[i]['score'] if valid_senti2[i]['label'] == 'NEG' else 0 for i in range(len(valid_senti2))]
valid_senti2_neu = [valid_senti2[i]['score'] if valid_senti2[i]['label'] == 'NEU' else 0 for i in range(len(valid_senti2))]
test_senti2_posi = [test_senti2[i]['score'] if test_senti2[i]['label'] == 'POS' else 0 for i in range(len(test_senti2))]
test_senti2_nega = [test_senti2[i]['score'] if test_senti2[i]['label'] == 'NEG' else 0 for i in range(len(test_senti2))]
test_senti2_neu = [test_senti2[i]['score'] if test_senti2[i]['label'] == 'NEU' else 0 for i in range(len(test_senti2))]

In [13]:
# no "neutral" while using sentiment1 
train_senti = pd.DataFrame()
train_senti['posi1'] = train_senti1_posi
train_senti['nega1'] = train_senti1_nega
train_senti['posi2'] = train_senti2_posi
train_senti['nega2'] = train_senti2_nega
train_senti['neu2'] = train_senti2_neu
train_senti['label'] = list(train['target'])
train_senti

,posi1,nega1,posi2,nega2,neu2,label
0,0.999164,0.000000,0.000000,0.000000,0.904307,1
1,0.000000,0.996901,0.000000,0.945229,0.000000,0
2,0.000000,0.974419,0.000000,0.534346,0.000000,1
3,0.000000,0.999123,0.000000,0.939632,0.000000,1
4,0.000000,0.833212,0.000000,0.000000,0.618091,0
...,...,...,...,...,...,...
6085,0.996390,0.000000,0.733445,0.000000,0.000000,0
6086,0.000000,0.999553,0.000000,0.956925,0.000000,0
6087,0.000000,0.993099,0.000000,0.000000,0.880959,0
6088,0.000000,0.992216,0.000000,0.000000,0.924359,1


In [14]:
valid_senti = pd.DataFrame()
valid_senti['posi1'] = valid_senti1_posi
valid_senti['nega1'] = valid_senti1_nega
valid_senti['posi2'] = valid_senti2_posi
valid_senti['nega2'] = valid_senti2_nega
valid_senti['neu2'] = valid_senti2_neu
valid_senti['label'] = list(valid['target'])

In [15]:
test_senti = pd.DataFrame()
test_senti['posi1'] = test_senti1_posi
test_senti['nega1'] = test_senti1_nega
test_senti['posi2'] = test_senti2_posi
test_senti['nega2'] = test_senti2_nega
test_senti['neu2'] = test_senti2_neu

In [19]:
train_senti.to_csv('data/train_senti.csv', index=False)
valid_senti.to_csv('data/valid_senti.csv', index=False)
test_senti.to_csv('data/test_senti.csv', index=False)

### Simplest lr model, just using sentimental scores from senti1(basic) and senti2(tweets)

In [8]:
from sklearn.linear_model import LogisticRegression

In [17]:
X = np.array(train_senti)
X_train = X[:, :5]
y_train = X[:, 5]
Xv = np.array(test_senti)
X_valid = X[:, :5]
y_valid = X[:, 5]
X_test = np.array(test_senti)

In [22]:
lr = LogisticRegression(random_state=0).fit(X_train, y_train)
lr.score(X_valid, y_valid)

0.5678160919540229

### add sklearn tokenized features 

In [29]:
train_lr = pd.read_csv("data/train_lr.csv")
valid_lr = pd.read_csv("data/valid_lr.csv")
test_lr = pd.read_csv("data/test_lr.csv")
train_senti = pd.read_csv("data/train_senti.csv")
valid_senti = pd.read_csv("data/valid_senti.csv")
test_senti = pd.read_csv("data/test_senti.csv")

In [3]:
y_train = train_lr['target']
y_valid = valid_lr['target']

In [12]:
X_train = train_lr.drop('target', axis = 1)
#X_train['posi1'] = train_senti['posi1']
#X_train['nega1'] = train_senti['nega1']
X_train['posi2'] = train_senti['posi2']
X_train['nega2'] = train_senti['nega2']
X_train['neu2'] = train_senti['neu2']
X_train = np.array(X_train)

In [25]:
X_valid = valid_lr.drop('target', axis = 1)
#X_valid['posi1'] = valid_senti['posi1']
#X_valid['nega1'] = valid_senti['nega1']
X_valid['posi2'] = valid_senti['posi2']
X_valid['nega2'] = valid_senti['nega2']
X_valid['neu2'] = valid_senti['neu2']
X_valid = np.array(X_valid)

In [33]:
X_test = test_lr
X_test['posi2'] = test_senti['posi2']
X_test['nega2'] = test_senti['nega2']
X_test['neu2'] = test_senti['neu2']
X_test = np.array(X_test)

In [31]:
lr = LogisticRegression(random_state=0).fit(X_train, y_train)
lr.score(X_valid, y_valid)

0.799080761654629

In [34]:
pred = lr.predict(X_test)
len(pred)

3263

In [35]:
test_data = pd.read_csv("data/test.csv")
preds_df = pd.DataFrame({"id": test_data["id"], "target": pred})
preds_df.to_csv(f"data/output_senti_lr.csv", index=False)